# **Section 4.2.1 (Gauss-Bonnet implications)**

## **min_surface_index**

- *min_surface_index* below takes an edge labeling $orb = [a_1, a_2, ..., a_9]$ corresponding to a prism orbifold (disregarding the infinite families)
-  It returns the minimum index in **Corollary 4.13**.


In [ ]:
from fractions import Fraction
import sympy as sp

def min_surface_index(orb):
    a, b, c = orb[3:6] # a, b, c correspond to a_4, a_5, a_6 in our labeling
    min_index = None
    triangle_area = (1 - (Fraction(1, a) + Fraction(1, b) + Fraction(1, c)))
    polygon_area = 2 * a * triangle_area # area of doubled cross section
    k = 4 # since min surface are for genus 2 is 4*pi
    while Fraction(k, polygon_area).denominator != 1:
      k += 4 # moving to the next smallest surface area, increment genus g
    min_index = Fraction(k, polygon_area).numerator  * a
    return min_index

# **Section 4.2.2 (Vertex group degrees)**

## **vertex_group_lcm**

- *vertex_group_lcm* is a function that takes an edge labeling $orb = [a_1, a_2, ..., a_9]$ corresponding to a prism orbifold (disregarding the infinite families)
- It returns the least common multiple of the vertex group degrees of thge orbifold.

In [ ]:
from fractions import Fraction
import math

def vertex_group_lcm(orb):
    v = [0]*6 # each element corresponds to a non ideal vertex vertex
    v[1] = [orb[0], orb[2], orb[3]] # edge labels associated to vertex 1
    v[2] = [orb[1], orb[2], orb[5]] # edge labels associated to vertex 2
    v[3] = [orb[3], orb[6], orb[8]] # edge labels associated to vertex 3
    v[4] = [orb[4], orb[6], orb[7]] # edge labels associated to vertex 4
    v[5] = [orb[5], orb[7], orb[8]] # edge labels associated to vertex 5
    # list of min indices associated to v[1],...,v[5]
    min_index = [1,1,1,1,1] 
    for i in range(1,6):
      # area of spherical triangle at a vertex embedded in sphere of radius 1
      triangle_area = ((Fraction(1, v[i][0]) + Fraction(1, v[i][1]) + Fraction(1, v[i][2]))-1)
      # number of copies of doubled triangle in triangulation of the sphere
      min_index[i-1]=int(2/triangle_area) 
    return math.lcm(*min_index)

# **Section 4.3 (Low index subgroups)**

- The following snippets of code, use the *Low Index Subgroups version 1.1.0* which is a Python module that enumerates subgroups of a finitely presented group up to a certain prescribed index. 
- It can produce the generators of these subgroups in terms of permutation representations. This is achived using the command `group.permutation-rep()`.
- This module was developed by Marc Culler, Matthias Goerner, and Nathan Dunfield. See the [Low Index GitHub Repository](https://github.com/3-manifolds/low_index) of the project for more information.
- In order to install the version of the module that we used simply run: `pip install low-index==1.1.0 `



## Covers of $O^{333}_{2}$
- Recall that the fundamental group of the double cover of $O^{333}_{2}$ is

    $$\big< x, y, z, w \mid x^3, y^3, z^3, w^2, (y^{-1}x)^2, (z^{-1}x)^3, (z^{-1}y)^4, (y^{-1}w)^3, (z^{-1}w)^2 \big>$$
    
- We were able to enumerate the subgroups of this group up to index $24$.
- **Note:** The index 24 was chosen since it is the minimum index for a potential manifold cover that could be a knot complement (see **Theorem 4.3**).
- Theoretically we can look at higher integer multiples of 24, but the computations proved to be extremely heavy.
- Note that even though $7$ is the number of generators, we essentially still have the four generators $x$, $y$, $z$, $y$, as
    - $a$ and $b$ correspond to $x$ and $x^{-1}$,
    - $c$ and $d$ correspond to $y$ and $y^{-1}$,
    - $e$ and $f$ correspond to $z$ and $z^{-1}$, 
    - $g$ corresponds to $w$
- This results in a txt file that you can find here under the name **raw_O333-2_up_to_indx_24.txt**

In [ ]:
from low_index import *

idx = 24
t = SimsTree(7, idx, ['aaa','ccc','eee','gg','ab','cd','ef','dada','fafafa','fcfcfcfc','gcgcgc','gege'])
sgrps = t.list()

# Writing out the results
with open(f"raw_O333-2_up_to_indx_{idx}.txt", "w") as file:
    # Write each number in the list to the file
    file.write(f"number of subgroups of O333-2 w index up to {idx} : {len(sgrps)} \n")
    for item in sgrps:
        file.write(str(item.permutation_rep()) + "\n")

### Data Cleaning
- Since we have 7 generators in the *SimsTree* presentation above, we have 7 generators for each subgroup. 
- For instance one of the index 3 subgroups is a list containing the following generators:
                
        [0, 1, 2]
        [0, 1, 2]
        [0, 2, 1]
        [0, 2, 1]
        [1, 2, 0]
        [2, 0, 1]
        [1, 0, 2]

- We delete the generators corresponding to $x^{-1}$, $y^{-1}$, and $z^{-1}$, that is indices 1, 3, and 5 in each list.
- We then seprate the subgroups of index exactly 24.
- These are stored here in a txt file **O333-2_indx_24.txt**

In [ ]:
# Here "permreps" is a list of all the subgroups as in the raw data
# Funtion that loads the raw data into a Python array
def load_txt_into_array(file_path):
    permreps = []
    with open(file_path, 'r') as infile:
        # Skip the first line
        next(infile) # Ignoring the first line of txt file as it contains the information about the number of subgroups
        for line in infile:
            # Evaluate the string representation of the line to convert it into a list of lists
            lists = eval(line.strip())
            permreps.append(lists)
    return permreps

file_path = '/path/to/raw_O333-2_up_to_indx_24.txt'  # Replace with your actual "raw_O333-2_up_to_indx_24.txt" file path
permreps = load_txt_into_array(file_path)

# Removing the redundant generators
indexes = [1, 3, 5]
for permrep in permreps:
    for index in sorted(indexes, reverse=True):
        del permrep[index]

# Grabbing the subgroups with index exactly 24
cleaned_permreps = []
for permrep in permreps:
    if len(permrep[0]) == 24:
        cleaned_permreps.append(permrep)

# Storing the cleaned permutation representations
with open("O333-2_indx_24.txt", "w") as file:
    for permrep in cleaned_permreps:
        file.write(str(permrep) + "\n")

## Covers of $O^{333}_{3}$
- Recall that the fundamental group of the double cover of $O^{333}_{3}$ is

    $$\big< x, y, z, w \mid x^3, y^3, z^3, w^2, (y^{-1}x)^2, (z^{-1}x)^3, (z^{-1}y)^4, (y^{-1}w)^2, (z^{-1}w)^3 \big>$$
    
- We were able to enumerate the subgroups of this group up to index $24$.
- **Note:** The index 24 was chosen since it is the minimum index for a potential manifold cover that could be a knot complement (see **Theorem 4.3**).
- Theoretically we can look at higher integer multiples of 24, but the computations proved to be extremely heavy.
- Similar to the previous case, even though $7$ is the number of generators, we essentially still have the four generators $x$, $y$, $z$, $y$, as
    - $a$ and $b$ correspond to $x$ and $x^{-1}$,
    - $c$ and $d$ correspond to $y$ and $y^{-1}$,
    - $e$ and $f$ correspond to $z$ and $z^{-1}$, 
    - $g$ corresponds to $w$
- This results in a txt file that you can find here under the name **raw_O333-3_up_to_indx_24.txt**

In [ ]:
from low_index import *

idx = 24
t = SimsTree(7, idx, ['aaa','ccc','eee','gg','ab','cd','ef','dada','fafafa','fcfcfcfc','dgdg','fgfgfg'])
sgrps = t.list()

# Open the file in write mode
with open(f"raw_O333-3_up_to_indx_{idx}.txt", "w") as file:
    # Write each number in the list to the file
    file.write(f"number of subgroups of O333-3 w index up to {idx} : {len(sgrps)} \n")
    for item in sgrps:
        file.write(str(item.permutation_rep()) + "\n")

### Data Cleaning
- Since we have 7 generators in the *SimsTree* presentation above, we have 7 generators for each subgroup. 
- For instance one of the index 16 subgroups is a list containing the following generators:
                
        [0, 1, 2, 6, 8, 4, 7, 3, 5, 9, 12, 14, 13, 10, 15, 11] 
        [0, 1, 2, 7, 5, 8, 3, 6, 4, 9, 13, 15, 10, 12, 11, 14]
        [0, 1, 4, 3, 5, 2, 9, 6, 8, 7, 11, 14, 13, 15, 10, 12]
        [0, 1, 5, 3, 2, 4, 7, 9, 8, 6, 14, 10, 15, 12, 11, 13]
        [1, 2, 0, 5, 3, 4, 10, 13, 14, 7, 11, 6, 8, 9, 12, 15]
        [2, 0, 1, 4, 5, 3, 11, 9, 12, 13, 6, 10, 14, 7, 8, 15]
        [0, 3, 5, 1, 4, 2, 12, 13, 8, 15, 10, 14, 6, 7, 11, 9]

- We delete the generators corresponding to $x^{-1}$, $y^{-1}$, and $z^{-1}$, that is indices 1, 3, and 5 in each list.
- We then seprate the subgroups of index exactly 24.
- These are stored here in a txt file **O333-3_indx_24.txt**

In [ ]:
# The data loading and cleaning functions are exactly the same as the previous case
# Funtion that loads the raw data into a Python array
def load_txt_into_array(file_path):
    permreps = []
    with open(file_path, 'r') as infile:
        # Skip the first line
        next(infile) # Ignoring the first line of txt file as it contains the information about the number of subgroups
        for line in infile:
            # Evaluate the string representation of the line to convert it into a list of lists
            lists = eval(line.strip())
            permreps.append(lists)
    return permreps

file_path = '/path/to/raw_O333-3_up_to_indx_24.txt'  # Replace with your actual "raw_O333-3_up_to_indx_24.txt" file path
permreps = load_txt_into_array(file_path)

# Removing the redundant generators
indexes = [1, 3, 5]
for permrep in permreps:
    for index in sorted(indexes, reverse=True):
        del permrep[index]

# Grabbing the subgroups with index exactly 24
cleaned_permreps = []
for permrep in permreps:
    if len(permrep[0]) == 24:
        cleaned_permreps.append(permrep)

# Storing the cleaned permutation representations
with open("O333-3_indx_24.txt", "w") as file:
    for permrep in cleaned_permreps:
        file.write(str(permrep) + "\n")

## Covers of $O^{333}_{4}$
- Recall that the fundamental group of the double cover of $O^{333}_{4}$ is

    $$\big< x, y, z, w \mid x^3, y^3, z^3, w^3, (y^{-1}x)^2, (z^{-1}x)^3, (z^{-1}y)^4, (y^{-1}w)^2, (z^{-1}w)^2 \big>$$
    
- We were able to enumerate the subgroups of this group up to index $24$.
- **Note:** The index 24 was chosen since it is the minimum index for a potential manifold cover that could be a knot complement (see **Theorem 4.3**).
- Theoretically we can look at higher integer multiples of 24, but the computations proved to be extremely heavy.
- **Slightly different** than before in the low-index syntax presentation of the group we have 8 generators, which relate to $x$, $y$, $z$, $y$ as follows:
    - $a$ and $b$ correspond to $x$ and $x^{-1}$,
    - $c$ and $d$ correspond to $y$ and $y^{-1}$,
    - $e$ and $f$ correspond to $z$ and $z^{-1}$, 
    - $g$ and $h$ correspond to $w$ and $w^{-1}$,
- This results in a txt file that you can find here under the name **raw_O333-4_up_to_indx_24.txt**

In [ ]:
from low_index import *

idx = 24
t = SimsTree(8, idx, ['aaa','ccc','eee','ggg','ab','cd','ef', 'gh','dada','fafafa','fcfcfcfc','dgdg','fgfg'])
sgrps = t.list()

# Open the file in write mode
with open(f"raw_O333-4_up_to_indx_{idx}.txt", "w") as file:
    # Write each number in the list to the file
    file.write(f"number of subgroups of O333-4 w index up to {idx} : {len(sgrps)} \n")
    for item in sgrps:
        file.write(str(item.permutation_rep()) + "\n")

### Data Cleaning
- Since we have 8 generators in the *SimsTree* presentation above, we have 8 generators for each subgroup. 
- For instance one of the index 4 subgroups is a list containing the following generators:
                
        [0, 3, 1, 2]
        [0, 2, 3, 1]
        [1, 2, 0, 3]
        [2, 0, 1, 3]
        [0, 3, 1, 2]
        [0, 2, 3, 1]
        [3, 0, 2, 1]
        [1, 3, 2, 0]

- We delete the generators corresponding to $x^{-1}$, $y^{-1}$, $z^{-1}$, and $w^{-1}$, that is indices 1, 3, 5, and 7 in each list.
- We then seprate the subgroups of index exactly 24.
- These are stored here in a txt file **O333-4_indx_24.txt**

In [ ]:
# The data loading and cleaning functions are very simialr to the previous case
# The only difference is we need to delete an extra index
# Funtion that loads the raw data into a Python array
def load_txt_into_array(file_path):
    permreps = []
    with open(file_path, 'r') as infile:
        # Skip the first line
        next(infile) # Ignoring the first line of txt file as it contains the information about the number of subgroups
        for line in infile:
            # Evaluate the string representation of the line to convert it into a list of lists
            lists = eval(line.strip())
            permreps.append(lists)
    return permreps

file_path = '/path/to/raw_O333-4_up_to_indx_24.txt'  # Replace with your actual "raw_O333-4_up_to_indx_24.txt" file path
permreps = load_txt_into_array(file_path)

# Removing the redundant generators
indexes = [1, 3, 5, 7]
for permrep in permreps:
    for index in sorted(indexes, reverse=True):
        del permrep[index]

# Grabbing the subgroups with index exactly 24
cleaned_permreps = []
for permrep in permreps:
    if len(permrep[0]) == 24:
        cleaned_permreps.append(permrep)

# Storing the cleaned permutation representations
with open("O333-4_indx_24.txt", "w") as file:
    for permrep in cleaned_permreps:
        file.write(str(permrep) + "\n")